In [251]:
using Lux, Random, ComponentArrays, TaylorDiff

rng = MersenneTwister()
Random.seed!(rng, 1)

MersenneTwister(1)

In [92]:
const input = 2

2

In [113]:
model = Chain(Dense(input => hidden, exp),
    Dense(hidden => hidden, exp),
    Dense(hidden => 1),
    first)
#model = Chain(Dense(2 => 30, relu), Dense(30 => 30, relu), Dense(30 => 1))


Chain(
    layer_1 = Dense(2 => 16, exp),      # 48 parameters
    layer_2 = Dense(16 => 16, exp),     # 272 parameters
    layer_3 = Dense(16 => 1),           # 17 parameters
    layer_4 = WrappedFunction(first),
)         # Total: 337 parameters,
          #        plus 0 states.

In [114]:
ps, st = Lux.setup(Xoshiro(0), model)
ps = ps |> ComponentArray

ComponentVector{Float32}(layer_1 = (weight = Float32[-1.0811747 0.9120898; -0.10964308 -0.41804138; … ; -0.506154 1.0804072f-5; 1.1675979 -0.07285434], bias = Float32[0.2490593, -0.4806301, 0.6780589, 0.0016063007, -0.522484, -0.6805993, -0.21060704, 0.50937545, 0.33639288, 0.22010256, -0.12450862, 0.3884359, 0.5799375, 0.39842856, -0.6958851, 0.07831879]), layer_2 = (weight = Float32[0.28811473 0.36900693 … -0.37211332 0.019529067; 0.14447463 -0.24254143 … -0.29063964 0.09696433; … ; -0.39559042 0.25605455 … -0.063562825 -0.16985989; 0.1261293 -0.20957911 … 0.052887935 0.37911317], bias = Float32[-0.19076955, -0.029651344, 0.23485127, -0.21793231, 0.21854055, -0.024576545, -0.13121834, 0.101620615, 0.01067999, 0.07822749, 0.13287592, -0.03696522, -0.19945842, 0.22354248, 0.085831076, -0.14627987]), layer_3 = (weight = Float32[-0.4188601 -0.39381167 … 0.086255476 -0.30063578], bias = Float32[-0.13529116]), layer_4 = Float32[])

In [115]:
x = rand(Float32, input)
#x = collect(range(0f0, 5f0, length=50))'


2-element Vector{Float32}:
 0.70260113
 0.19438636

In [116]:
u = Lux.apply(model, x, ps, st)[1]

-28.973803f0

In [118]:
function trial(model, x, ps, st)
    u, st = Lux.apply(model, x, ps, st)
    return u
end

f(x) = trial(model, x, ps, st)

TaylorDiff. derivative(f, x, Float32[1, 0], Val(2))

-186.615f0

In [119]:
using Lux, Random, ComponentArrays, TaylorDiff

const input = 2

model = Chain(Dense(input => hidden, exp),
    Dense(hidden => hidden, exp),
    Dense(hidden => 1),
    first)

ps, st = Lux.setup(Xoshiro(0), model)
ps = ps |> ComponentArray    
x = rand(Float32, input)
u = Lux.apply(model, x, ps, st)[1]

function trial(model, x, ps, st)
    u, st = Lux.apply(model, x, ps, st)
    return u
end

f(x) = trial(model, x, ps, st)

TaylorDiff. derivative(f, x, Float32[1, 0], Val(2))

-35.121586f0

In [252]:
using Lux, Random, ComponentArrays, TaylorDiff

const input = 1
const hidden = 16  # Define the number of hidden units

# Define the model
model = Chain(Dense(1 => 30, relu), Dense(30 => 30, relu), Dense(30 => 1)) 

# Initialize model parameters
ps, st = Lux.setup(Xoshiro(0), model)
ps = ps |> ComponentArray

# Function to evaluate the model
function trial(model, x, ps, st)
    u, st = Lux.apply(model, x, ps, st)
    return u
end

f(x) = trial(model, x, ps, st)

 
x = collect(range(0f0, 5f0, 50))
x = reshape(x, 1, :)
y = reshape(Float32.(sin.(x)), 1, 50)
data = (x, y)


TaylorDiff.derivative(f, x, Float32.(ones(size(x))), Val(1)) 

1×50 Matrix{Float32}:
 -2.16463  0.73768  1.67917  0.561669  …  1.63091  1.63091  1.63818  1.66937

In [247]:
x = collect(range(0f0, 5f0, length=50))'
 

1×50 adjoint(::Vector{Float32}) with eltype Float32:
 0.0  0.102041  0.204082  0.306122  …  4.69388  4.79592  4.89796  5.0

In [237]:
x

1×2 Matrix{Float32}:
 1.0  1.0

In [241]:
x = collect(range(0f0, 5f0, length=50))
x = reshape(x, 1, :)

1×50 Matrix{Float32}:
 0.0  0.102041  0.204082  0.306122  …  4.69388  4.79592  4.89796  5.0

In [190]:
using Lux, Random, ComponentArrays, TaylorDiff

# Define the input and target
x = [2 0; 0 1]    # Generates a (2x10) input
target = rand(Float32, 1, 10)  # Target values with shape (1x10)

# Define the model
model = Chain(
    Dense(1 => 15, tanh),
    Dense(15 => 15, tanh),
    Dense(15 => 1)
)

# Initialize model parameters and state
ps, st = Lux.setup(Xoshiro(2), model)
ps = ps |> ComponentArray

# Function to evaluate the model
function trial(model, x, ps, st)
    u, st = Lux.apply(model, x, ps, st)
    return u
end

# Wrapper function for single input
f(x) = trial(model, x, ps, st)

# Compute the derivative of the sum of outputs with respect to input
TaylorDiff.derivative.(f,x,Val(1))

 


MethodError: MethodError: no method matching (::Dense{typeof(tanh), Int64, Int64, Nothing, Nothing, Static.True})(::TaylorScalar{Int64, 1}, ::ComponentVector{Float32, SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, Tuple{Axis{(weight = ViewAxis(1:15, ShapedAxis((15, 1))), bias = 16:30)}}}, ::@NamedTuple{})
The object of type `Dense{typeof(tanh), Int64, Int64, Nothing, Nothing, Static.True}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

Closest candidates are:
  (::Dense)(!Matched::AbstractArray, ::Any, ::NamedTuple)
   @ Lux ~/.julia/packages/Lux/CXGnc/src/layers/basic.jl:339


In [169]:
rand(Float32, 2, 10)

2×10 Matrix{Float32}:
 0.753373  0.928309  0.0260389  0.141545  …  0.117271  0.881823  0.886729
 0.551721  0.907624  0.77566    0.676511     0.207066  0.600407  0.225683

In [187]:
input = 1
hidden = 10
W₁, W₂, b₁, b₂ = rand(hidden, input), rand(1, hidden), rand(hidden), rand(1)
σ = exp
mlp(x) = (W₂ * σ.(W₁ * x + b₁) + b₂)
TaylorDiff.derivative.(mlp,x,Val(1))

2×10 Matrix{Matrix{Float64}}:
 [9.83895;;]  [16.5869;;]  [5.64471;;]  …  [5.348;;]    [22.5493;;]
 [5.25608;;]  [6.22501;;]  [6.94439;;]     [9.01024;;]  [4.98047;;]